In [ ]:
import pandas as pd
import numpy as np

raw_file = "/home/ec2-user/SageMaker/Team-5/Fiona/TN_allinone0112.csv"
df = pd.read_csv(raw_file)

print(df.shape)

## Converting heights
df['Heightin_to_cm'] = df['Heightin'] * 2.53999863
df['Height_cm'] = df['Heightin_to_cm'] + df['Heightcm']
df.insert(df.columns.get_loc('Height_cm') + 1, 'Height_cm_units', 'cm')
df.drop(['Heightin_to_cm', 'Heightcm', 'Heightin'], axis=1, inplace=True)

df['Weightlbs_to_kg'] = df['Weightlbs'] * 0.45359237
df['Weight_kg'] = df['Weightlbs_to_kg'] + df['Weightkg']
df.insert(df.columns.get_loc('Weight_kg') + 1, 'Weight_kg_units', 'kg')
df.drop(['Weightlbs_to_kg', 'Weightkg', 'Weightlbs'], axis=1, inplace=True)

## Converting glucose units and organizing columns
for column in df.columns:
    if 'mmol' in column.lower() or 'mgdl' in column.lower():
        new_column_name = column.replace('mmol', '_unit').replace('mgdl', '').replace('Mmoll', '_unit').replace('MgDL', '').replace('mmoll', '_unit')
        df.rename(columns={column: new_column_name}, inplace=True)
        
columns_to_process = ['AvgRecordedFastingGlucoses', 'AvgAllRecordedGlucoses', 'LowestGlucose', 'HighestGlucose']
df[columns_to_process] = df[columns_to_process].apply(lambda x: x.where(x >= 45, x * 18.018))

columns_to_convert = ['AvgRecordedFastingGlucoses_unit', 'AvgAllRecordedGlucoses_unit', 'LowestGlucose_unitl', 'HighestGlucose_unit']
df[columns_to_convert] = df[columns_to_convert].replace('mmol/L', 'mg/dl')

Hemoglobin_process = ['HemoglobinResults']
df[Hemoglobin_process] = df[Hemoglobin_process].apply(lambda x: x.where(x <= 30, x / 10))

Hematocrit_process = ['HematocritResults']
df[Hematocrit_process] = df[Hematocrit_process].apply(lambda x: x.where(x >= 1, x * 100))

column_to_move1 = 'AvgRecordedFastingGlucoses_unit'
target_position1 = 'AvgRecordedFastingGlucoses'
current_positions = df.columns.tolist()
current_positions.remove(column_to_move1)
target_index = current_positions.index(target_position1)
new_positions = current_positions[:target_index + 1] + [column_to_move1] + current_positions[target_index + 1:]
df = df[new_positions]

column_to_move2 = 'AvgAllRecordedGlucoses_unit'
target_position2 = 'AvgAllRecordedGlucoses'
current_positions = df.columns.tolist()
current_positions.remove(column_to_move2)
target_index = current_positions.index(target_position2)
new_positions = current_positions[:target_index + 1] + [column_to_move2] + current_positions[target_index + 1:]
df = df[new_positions]

df.replace('mg/dl', 'mg/dL', inplace=True, regex=True)

df.rename(columns={'LowestGlucose_unitl': 'LowestGlucose_unit'}, inplace=True)

print(df.shape)